# Setup

In [20]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

In [2]:
train = pd.read_json("_RawData/train.json/data/processed/train.json")
test = pd.read_json("_RawData/test.json/data/processed/test.json")

In [3]:
train.head()

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0


# CNN

In [35]:
def get_channel(band_data):
    print("get_channel")
    band_data = np.array([np.array(band).astype(np.float32) for band in band_data])
    
    print(band_data.shape)
    band_data_new = np.empty((1604, 150, 150))

    for ind in range(band_data.shape[0]):
        arr_image = band_data[ind]
        band_data_new[ind] = (scipy.misc.fromimage(scipy.misc.toimage(arr_image.reshape(75, 75)).resize((150, 150))))

    print(band_data_new.shape)

    return band_data_new

def concat_data(data):
    channel1 = get_channel(data["band_1"])
    channel2 = get_channel(data["band_2"])
    channel3 = (channel1 + channel2) / 2.

    return np.concatenate([channel1[:, :, :, np.newaxis], channel2[:, :, :, np.newaxis],channel3[:, :, :, np.newaxis]], axis=-1)

In [36]:
X_train = concat_data(train)

get_channel
(1604, 5625)
(1604, 150, 150)
get_channel
(1604, 5625)
(1604, 150, 150)


In [37]:
from sklearn.model_selection import train_test_split

y_train=train['is_iceberg']
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, y_train, random_state=1, train_size=0.75)

C:\Users\Dave\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\model_selection\_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [38]:
def get_generators(train_data, valid_data):
    data_gen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            rotation_range=180,
            vertical_flip=True,
            horizontal_flip=True)

    data_gen.fit(train_data)

    val_gen = ImageDataGenerator(rescale=1./255)
    val_gen.fit(valid_data)
    
    return data_gen, val_gen

In [39]:
dgen, vgen = get_generators(X_train_cv, X_valid)

In [46]:
from keras.applications import VGG16

vgg_model = VGG16(include_top=False, weights=None, input_shape=(150, 150, 3))

top_model = Sequential()
top_model.add(Flatten(input_shape=vgg_model.output_shape[1:]))
top_model.add(Dense(512, activation='relu'))
# top_model.add(Dense(512, activation='relu'))
# top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
# top_model.load_weights(top_model_weights_path)

model = Model(inputs= vgg_model.input, outputs= top_model(vgg_model.output))

model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [47]:
model.fit_generator(dgen.flow(X_train_cv, y_train_cv, batch_size=32),
                    steps_per_epoch=len(X_train) / 32, 
                    validation_data=vgen.flow(X_valid, y_valid, batch_size=32, shuffle=False),
                    validation_steps = len(X_valid) / 32,
                    epochs=10)

Epoch 1/10
51/50 [==============================] - 23s - loss: 0.6808 - acc: 0.5624 - val_loss: 0.6520 - val_acc: 0.5885
Epoch 2/10
51/50 [==============================] - 24s - loss: 0.5809 - acc: 0.7054 - val_loss: 0.5776 - val_acc: 0.7332
Epoch 3/10
51/50 [==============================] - 23s - loss: 0.5228 - acc: 0.7558 - val_loss: 0.4983 - val_acc: 0.7980
Epoch 4/10
51/50 [==============================] - 25s - loss: 0.4908 - acc: 0.7798 - val_loss: 0.5023 - val_acc: 0.7880
Epoch 5/10
51/50 [==============================] - 25s - loss: 0.4798 - acc: 0.7839 - val_loss: 0.4766 - val_acc: 0.7756
Epoch 6/10
51/50 [==============================] - 24s - loss: 0.4898 - acc: 0.7771 - val_loss: 0.4622 - val_acc: 0.8005
Epoch 7/10
51/50 [==============================] - 24s - loss: 0.4790 - acc: 0.7837 - val_loss: 0.4463 - val_acc: 0.8080
Epoch 8/10
51/50 [==============================] - 25s - loss: 0.4810 - acc: 0.7808 - val_loss: 0.4507 - val_acc: 0.7980
Epoch 9/10
51/50 [======